# Métricas do reprocessamento

Código feito por Igor Laltuf, set/23

---

## Objetivo

Analisar o impacto do reprocessamento das viagens das linhas LECD50/605 entre 01/02/2023 até 30/09/2023 (aguardando resultados de outubro para reprocessar)

## Etapas
Comparar valores pré e pós reprocessamento segundo as seguintes variáveis:
- A quantidade de viagens completas
- Quilometragem total de viagens completas
- Valor do subsídio a ser pago
- Explicações pontuais sobre os resultados encontrados

## Conclusão

Sobre o período entre 01/02/2023 e 30/09/2023:

1) A linha LECD50 não teve viagens planejadas no mês de janeiro de 2023.


**REESCREVER OS TÓPICOS ABAIXO**



2) A quilometragem total das viagens completas aumentou de 66.790,1 km para 81.485,2 km, um aumento de 22% após o reprocessamento.

3) A quantidade de viagens identificadas aumentou de 6.414 para 15.670 viagens. Como as viagens que antes eram circulares foram divididas em duas viagens (ida e volta), era esperado que a quantidade de viagens pelo menos dobrasse, caso nenhuma nova viagem fosse identificada.

4) O valor do subsídio antes do reprocessamento era de R$ 100.756,68 e após o reprocessamento foi de R$ 169.626,86. Vale destacar que o valor do reprocessamento foi calculado sem os descontos por km.

### Carregar bibliotecas

In [2]:
#!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import folium
from datetime import timedelta, datetime
from shapely import wkt

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve()

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'

for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"

### 1. Comparar a variação na km após o reprocessamento

In [3]:
q = f"""
SELECT
  FORMAT_DATETIME('%Y-%m', DATA) AS mes_ano,
  SUM(ROUND(distancia_planejada,2)) AS distancia_total
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_completa`
WHERE
  DATA BETWEEN "2023-02-01"
  AND "2023-09-30"
  AND servico_informado IN ("LECD50","605")
GROUP BY
  mes_ano
"""

viagens_realizadas = bd.read_sql(q, from_file=True)

viagens_realizadas.to_csv('../data/treated/viagens_realizadas.csv', index=False)

Downloading: 100%|██████████| 8/8 [00:00<00:00, 19.42rows/s]


In [4]:
q = f"""
SELECT
  FORMAT_DATETIME('%Y-%m', DATA) AS mes_ano,
  SUM(ROUND(distancia_planejada,2)) AS distancia_total
FROM
  `rj-smtr-dev.SMTR202303002183_reprocessamento.viagem_completa` 
WHERE
  DATA BETWEEN "2023-02-01"
  AND "2023-09-30"
  AND servico_informado IN ("LECD50","605")
GROUP BY
  mes_ano
"""
viagens_reprocessadas = bd.read_sql(q, from_file=True)

viagens_reprocessadas.to_csv('../data/treated/viagens_reprocessadas.csv', index=False)

Downloading: 100%|██████████| 8/8 [00:00<00:00, 19.56rows/s]


In [5]:
viagens_reprocessadas = pd.read_csv('../data/treated/viagens_reprocessadas.csv')
viagens_realizadas = pd.read_csv('../data/treated/viagens_realizadas.csv')

km_mes_ano = pd.merge(viagens_realizadas, viagens_reprocessadas, how = 'left', on='mes_ano', suffixes=['','_reprocessada'])
km_mes_ano = km_mes_ano.sort_values('mes_ano')
km_mes_ano

,mes_ano,distancia_total,distancia_total_reprocessada
3,2023-02,108.90,2570.04
2,2023-03,65.34,2428.47
5,2023-04,435.60,522.72
0,2023-05,577.17,1001.88
6,2023-06,827.64,1121.67
4,2023-07,947.43,1045.44
7,2023-08,1012.77,1067.22
1,2023-09,620.73,892.98


In [6]:
distancia_total_pre_reprocessamento = km_mes_ano['distancia_total'].sum()
distancia_total_pos_reprocessamento = km_mes_ano['distancia_total_reprocessada'].sum()
print("Comparação da quilometragem das viagens completas")
print("Valor total da km pré-reprocessamento:", distancia_total_pre_reprocessamento)
print("Valor total da km pós-reprocessamento:", distancia_total_pos_reprocessamento)
print("Aumento de ",round((distancia_total_pos_reprocessamento / distancia_total_pre_reprocessamento - 1) * 100, 2),"%")

Comparação da quilometragem das viagens completas
Valor total da km pré-reprocessamento: 4595.58
Valor total da km pós-reprocessamento: 10650.42
Aumento de  131.75 %


In [7]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionar a linha com pontos para km_mes_ano['distancia_total']
fig.add_trace(go.Scatter(x=km_mes_ano['mes_ano'], y=km_mes_ano['distancia_total'],
                         mode='lines+markers',
                         name='Km pré-reprocessamento',
                         marker=dict(color='dodgerblue'),
                         opacity=0.75))

# Adicionar a linha com pontos para km_mes_ano['distancia_total_reprocessada']
fig.add_trace(go.Scatter(x=km_mes_ano['mes_ano'], y=km_mes_ano['distancia_total_reprocessada'],
                         mode='lines+markers',
                         name='Km reprocessada',
                         marker=dict(color='crimson'),
                         opacity=0.75))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Quilometragem apurada por mês - Serviço: LECD50 / 605", font=dict(color='black')),
    showlegend=False,
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valores',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.025,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pré-reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.27,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.29,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pós-reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

fig.show()

### 2. Comparar a variação no valor do subsídio após o reprocessamento

2.1 Valor recebido pelos consórcios antes do reprocessamento

In [8]:
q = f"""
SELECT
  CONCAT(CAST(EXTRACT(YEAR FROM DATA) AS STRING), '-', LPAD(CAST(EXTRACT(MONTH FROM DATA) AS STRING), 2, '0')) AS mes_ano,
  SUM(viagens) as viagens,
  
  SUM(valor_subsidio_pago) AS valor_subsidio
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico IN ('LECD50','605')
  AND DATA BETWEEN '2023-02-01' AND '2023-09-30'
GROUP BY
  mes_ano
"""
subsidio_pago_mes = bd.read_sql(q, from_file=True)


Downloading: 100%|██████████| 8/8 [00:00<00:00, 19.00rows/s]


In [9]:
subsidio_pago_mes = subsidio_pago_mes.sort_values('mes_ano')
subsidio_pago_mes = subsidio_pago_mes[['mes_ano', 'viagens', 'valor_subsidio']]
subsidio_pago_mes

,mes_ano,viagens,valor_subsidio
2,2023-02,10,0.00000
5,2023-03,6,0.00000
1,2023-04,40,674.01000
3,2023-05,53,453.12000
4,2023-06,74,1601.05000
6,2023-07,87,1927.84000
7,2023-08,93,2570.24000
0,2023-09,57,1162.71798


2.2 Valor que deve ser pago após o reprocessamento

Com base nas viagens identificadas na tabela de viagem completa que foi reprocessada, foram verificados aqueles dias em que POD >= 80%, para então calcular o valor do subsídio de acordo com os valores por km da tabela `subsidio_data_versao_efetiva` que está no dataset `rj-smtr-dev.SMTR202212006611_reprocessamento`.

Vale destacar que esta tabela exclui as glosas de 2023 do valor do subsídio por km. Ex: considera 2.81 por km após 16 de janeiro de 2023. 

In [10]:
q = f"""
WITH
  viagem_dia AS (
  SELECT
    DATA,
    COUNT(*) AS viagens,
    SUM(distancia_planejada) AS km_apurada
  FROM
    `rj-smtr-dev.SMTR202303002183_reprocessamento.viagem_completa`
  WHERE
    DATA BETWEEN '2023-02-01' 
    AND '2023-09-30'
  GROUP BY
    DATA
  ),
  viagem_planejada AS ( -- pegar a km planejada para o dia
  SELECT DISTINCT 
    DATA,
    distancia_total_planejada
  FROM
    `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
  WHERE
    servico IN ('LECD50','605')
    AND DATA BETWEEN '2023-02-01' 
    AND '2023-09-30'
  ),
  tabela_pod AS (
  SELECT
    *,
    COALESCE(ROUND((km_apurada / NULLIF(distancia_total_planejada, 0)), 2), 0) AS perc_km_planejada
  FROM
    viagem_dia vd
  RIGHT JOIN
    viagem_planejada vp
  USING
    (DATA) 
  ),
  tabela_subsidio_km AS (
  SELECT
    DATA,
    valor_subsidio_por_km
  FROM
    `rj-smtr-dev.SMTR202303002183_reprocessamento.subsidio_data_versao_efetiva` )
SELECT
  DATA,
  COALESCE(viagens, 0) as viagens,
  COALESCE(km_apurada,0) as km_apurada,
  COALESCE(distancia_total_planejada,0) as distancia_total_planejada,
  COALESCE(perc_km_planejada,0) as perc_km_planejada,
  COALESCE(valor_subsidio_por_km,0) as valor_subsidio_por_km,
   CASE 
   WHEN perc_km_planejada >= 0.8 THEN valor_subsidio_por_km * km_apurada
   ELSE 0
   END as valor_subsidio
FROM
  tabela_pod
LEFT JOIN
  tabela_subsidio_km
USING
  (DATA)
"""

subsidio_a_pagar_dia = bd.read_sql(q, from_file=True)
subsidio_a_pagar_dia = subsidio_a_pagar_dia.sort_values('DATA')
subsidio_a_pagar_dia

Downloading: 100%|██████████| 214/214 [00:00<00:00, 563.16rows/s]


,DATA,viagens,km_apurada,distancia_total_planejada,perc_km_planejada,valor_subsidio_por_km,valor_subsidio
154,2023-02-01,15,163.380,163.38,1.00,2.81,459.09780
28,2023-02-02,9,98.028,163.38,0.60,2.81,0.00000
50,2023-02-03,16,174.272,163.38,1.07,2.81,489.70432
169,2023-02-04,0,0.000,0.00,0.00,2.81,0.00000
16,2023-02-05,0,0.000,0.00,0.00,2.81,0.00000
...,...,...,...,...,...,...,...
137,2023-09-25,5,54.445,43.56,1.25,2.81,152.99045
140,2023-09-26,0,0.000,43.56,0.00,2.81,0.00000
42,2023-09-27,6,65.334,43.56,1.50,2.81,183.58854
135,2023-09-28,6,65.334,43.56,1.50,2.81,183.58854


In [11]:
subsidio_a_pagar_dia['DATA'] = pd.to_datetime(subsidio_a_pagar_dia['DATA'])
subsidio_a_pagar_dia['mes_ano'] = subsidio_a_pagar_dia['DATA'].dt.strftime('%Y-%m')
subsidio_a_pagar_mes = subsidio_a_pagar_dia[['mes_ano', 'viagens', 'valor_subsidio']]
subsidio_a_pagar_mes = subsidio_a_pagar_mes.groupby('mes_ano').sum().reset_index()  
subsidio_a_pagar_mes

,mes_ano,viagens,valor_subsidio
0,2023-02,236,6947.68004
1,2023-03,223,5815.23880
2,2023-04,48,1469.11296
3,2023-05,92,2693.37376
4,2023-06,103,3030.04548
5,2023-07,96,2937.76227
6,2023-08,98,2906.81855
7,2023-09,82,2509.04338


O aumento do valor do subsídio não é proporcional ao aumento da quantidade de viagens, pois com o reprocessamento, cada viagem circular virou duas viagens separadas, uma de ida e outra de volta.

Isto quer dizer que o esperado era que a quantidade de viagens pelo menos dobrasse após o reprocessamento, mesmo que não fosse identificada nenhuma viagem nova.

Valor do subsídio a ser pago por dia após o reprocessamento

In [12]:
q = f"""
SELECT
  *
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico IN ('LECD50','605')
  AND DATA BETWEEN '2023-02-01' 
  AND '2023-09-30'
"""
subsidio_pago_dia = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 166/166 [00:00<00:00, 393.36rows/s]


In [15]:
subsidio_pago_dia = subsidio_pago_dia[['data', 'viagens', 'valor_subsidio_pago']]
subsidio_pago_dia

subsidio_pago_dia = subsidio_pago_dia.rename(columns={
    'viagens': 'viagens_pre',
    'valor_subsidio_pago': 'subsidio_pre'
})

subsidio_pago_dia['subsidio_pre'] = subsidio_pago_dia['subsidio_pre'].round(2)

subsidio_pago_dia


KeyError: "['viagens', 'valor_subsidio_pago'] not in index"

In [17]:
subsidio_a_pagar_dia_formatado = subsidio_a_pagar_dia[['DATA', 'viagens', 'valor_subsidio']]

subsidio_a_pagar_dia_formatado = subsidio_a_pagar_dia_formatado.rename(columns={
    'DATA': 'data',
    'viagens': 'viagens_pos',
    'valor_subsidio': 'subsidio_pos'
})

subsidio_a_pagar_dia_formatado['subsidio_pos'] = subsidio_a_pagar_dia_formatado['subsidio_pos'].round(2)

# Certificando que 'data' é do tipo datetime em ambos os DataFrames
subsidio_a_pagar_dia_formatado['data'] = pd.to_datetime(subsidio_a_pagar_dia_formatado['data'])

subsidio_pago_dia['data'] = pd.to_datetime(subsidio_pago_dia['data'])

resultado = pd.merge(subsidio_a_pagar_dia_formatado, subsidio_pago_dia, on='data', how='outer')

resultado['subsidio_a_pagar'] = resultado['subsidio_pos'] - resultado['subsidio_pre']

resultado.to_excel('./../data/output/subsidio_a_pagar_dia.xlsx', index=False)

In [18]:
# juntar as duas tabelas
tabela_subsidio_mes = pd.merge(subsidio_pago_mes, subsidio_a_pagar_mes, how='left', on='mes_ano', suffixes=['_pre','_pos'])
tabela_subsidio_mes

,mes_ano,viagens_pre,valor_subsidio_pre,viagens_pos,valor_subsidio_pos
0,2023-02,10,0.00000,236,6947.68004
1,2023-03,6,0.00000,223,5815.23880
2,2023-04,40,674.01000,48,1469.11296
3,2023-05,53,453.12000,92,2693.37376
4,2023-06,74,1601.05000,103,3030.04548
5,2023-07,87,1927.84000,96,2937.76227
6,2023-08,93,2570.24000,98,2906.81855
7,2023-09,57,1162.71798,82,2509.04338


In [19]:
tabela_subsidio_mes
total_subsidio_comparacao = tabela_subsidio_mes[['viagens_pre', 'viagens_pos','valor_subsidio_pre','valor_subsidio_pos']].sum(axis=0)
total_subsidio_comparacao

viagens_pre             420.00000
viagens_pos             978.00000
valor_subsidio_pre     8388.97798
valor_subsidio_pos    28309.07524
dtype: float64

In [20]:
# Criar o gráfico de linha com plotly
fig = go.Figure()

# Adicionar a linha com pontos para tabela_subsidio_mes['viagens_pre']
fig.add_trace(go.Scatter(x=tabela_subsidio_mes['mes_ano'], y=tabela_subsidio_mes['viagens_pre'],
                         mode='lines+markers',
                         name='Qtd de viagens pré-reprocessamento',
                         marker=dict(color='dodgerblue'),
                         opacity=0.75))

# Adicionar a linha com pontos para tabela_subsidio_mes['viagens_pos']
fig.add_trace(go.Scatter(x=tabela_subsidio_mes['mes_ano'], y=tabela_subsidio_mes['viagens_pos'],
                         mode='lines+markers',
                         name='Qtd de viagens reprocessadas',
                         marker=dict(color='crimson'),
                         opacity=0.75))

# Configurar layout e outros parâmetros do gráfico seguindo o design anterior
fig.update_layout(
    title=dict(text="Quantidade de viagens por mês - Serviço: LECD50 / 605", font=dict(color='black')),
    showlegend=False,
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valores',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.025,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pré-reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.27,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.29,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pós-reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

fig.show()

In [21]:
# Criar o gráfico de linha com plotly
fig = go.Figure()

# Adicionar a linha com pontos para tabela_subsidio_mes['valor_subsidio_pre']
fig.add_trace(go.Scatter(x=tabela_subsidio_mes['mes_ano'], y=tabela_subsidio_mes['valor_subsidio_pre'],
                         mode='lines+markers',
                         name='Valor do subsídio pré-reprocessamento',
                         marker=dict(color='dodgerblue'),
                         opacity=0.75))

# Adicionar a linha com pontos para tabela_subsidio_mes['valor_subsidio_pos']
fig.add_trace(go.Scatter(x=tabela_subsidio_mes['mes_ano'], y=tabela_subsidio_mes['valor_subsidio_pos'],
                         mode='lines+markers',
                         name='Valor do subsídio reprocessado',
                         marker=dict(color='crimson'),
                         opacity=0.75))

# Configurar layout e outros parâmetros do gráfico seguindo o design anterior
fig.update_layout(
    title=dict(text="Valor do subsídio por mês - Serviço: LECD50 / 605", font=dict(color='black')),
    showlegend=False,
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valores',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.025,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pré-reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.27,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.29,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pós-reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

fig.show()

### 3. Análise do POD


3.1 POD pré-reprocessamento

In [22]:
# POD pré-reprocessamento
q = f"""
SELECT
  data, viagens, perc_km_planejada
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico IN ('LECD50','605')
  AND DATA BETWEEN '2023-02-01' 
  AND '2023-09-30'
"""
pod_pre_reprocessamento = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 166/166 [00:00<00:00, 407.86rows/s]


In [23]:
# Assegurando que a coluna 'data' seja do tipo datetime
pod_pre_reprocessamento = pod_pre_reprocessamento.sort_values('data')
pod_pre_reprocessamento['data'] = pd.to_datetime(pod_pre_reprocessamento['data'])
# Criando a nova coluna 'mes_ano'
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['data'].dt.to_period('M')
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['mes_ano'].astype(str)
pod_pre_reprocessamento

,data,viagens,perc_km_planejada,mes_ano
70,2023-02-01,1,6.67,2023-02
60,2023-02-02,0,0.00,2023-02
69,2023-02-03,1,6.67,2023-02
89,2023-02-06,1,6.67,2023-02
4,2023-02-07,0,0.00,2023-02
...,...,...,...,...
52,2023-09-25,3,74.99,2023-09
31,2023-09-26,0,0.00,2023-09
8,2023-09-27,5,124.99,2023-09
66,2023-09-28,4,99.99,2023-09


In [24]:

y = "perc_km_planejada"
title = f"Distribuição do percentual de operação diária (POD) por mês - Serviço: LECD50 / 605"

fig = px.box(
    pod_pre_reprocessamento, 
    x="mes_ano", 
    y=y, 
    title=title,
    template="plotly_white",
    labels={
        "mes_ano": "",
        y: ""
    },
    points="all"
)

fig.update_layout(
    yaxis_range=[-10, 200],  
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    showlegend=False,
)

fig.add_hline(y=80, annotation_text="min = 80%")  # Linha horizontal para indicar o mínimo de 80%

fig.show()


3.2 POD pós-reprocessamento

In [25]:
pod_pos_reprocessamento = subsidio_a_pagar_dia[["mes_ano","viagens","perc_km_planejada"]]
pod_pos_reprocessamento['perc_km_planejada'] = pod_pos_reprocessamento['perc_km_planejada'] * 100
pod_pos_reprocessamento

,mes_ano,viagens,perc_km_planejada
154,2023-02,15,100.0
28,2023-02,9,60.0
50,2023-02,16,107.0
169,2023-02,0,0.0
16,2023-02,0,0.0
...,...,...,...
137,2023-09,5,125.0
140,2023-09,0,0.0
42,2023-09,6,150.0
135,2023-09,6,150.0


In [26]:
y = "perc_km_planejada"
title = "Distribuição do percentual de operação diária (POD) por mês - Serviço: LECD50 / 605 <br>Versão reprocessada"

fig = px.box(
    pod_pos_reprocessamento, 
    x="mes_ano", 
    y=y, 
    title=title,
    template="plotly_white",
    labels={
        "mes_ano": "",
        y: ""
    },
    points="all"
)

fig.update_layout(
    yaxis_range=[-50, 230],  
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    showlegend=False,
)

fig.add_hline(y=80, annotation_text="min = 80%")  # Linha horizontal para indicar o mínimo de 80%

fig.show()


Consolidar POD pré e pós reprocessamento em apenas um gráfico.

In [27]:
# Calcular a média e o desvio padrão de perc_km_planejada por mês para ambos os DataFrames
media_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].mean()
std_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].std()

media_pos = pod_pos_reprocessamento.groupby('mes_ano')['perc_km_planejada'].mean()
std_pos = pod_pos_reprocessamento.groupby('mes_ano')['perc_km_planejada'].std()

Com o reprocessamento, houve uma melhoria do valor do Percentual de Operação Diária.

In [28]:
maiores_perc_pos = subsidio_a_pagar_dia.sort_values(by='perc_km_planejada', ascending=False)
maiores_perc_pos

,DATA,viagens,km_apurada,distancia_total_planejada,perc_km_planejada,valor_subsidio_por_km,valor_subsidio,mes_ano
135,2023-09-28,6,65.334,43.56,1.50,2.81,183.58854,2023-09
42,2023-09-27,6,65.334,43.56,1.50,2.81,183.58854,2023-09
155,2023-09-20,6,65.334,43.56,1.50,2.81,183.58854,2023-09
184,2023-03-02,20,217.840,163.38,1.33,2.81,612.13040,2023-03
144,2023-05-19,5,54.460,43.57,1.25,2.81,153.03260,2023-05
...,...,...,...,...,...,...,...,...
85,2023-04-18,0,0.000,43.56,0.00,2.81,0.00000,2023-04
113,2023-06-25,0,0.000,0.00,0.00,2.81,0.00000,2023-06
109,2023-06-24,0,0.000,0.00,0.00,2.81,0.00000,2023-06
97,2023-04-19,0,0.000,43.56,0.00,2.81,0.00000,2023-04


In [29]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionar a linha com pontos e desvio padrão para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=media_pre.index, y=media_pre.values,
                    mode='lines+markers',
                    name='Antes do Reprocessamento',
                    marker=dict(color='dodgerblue'),
                    opacity=0.75,
                    error_y=dict(type='data', array=std_pre, visible=True, color='dodgerblue')  # Corrigindo a cor aqui
))

# Adicionar a linha com pontos e desvio padrão para pod_pos_reprocessamento
fig.add_trace(go.Scatter(x=media_pos.index, y=media_pos.values,
                    mode='lines+markers',
                    name='Após Reprocessamento',
                    marker=dict(color='crimson'),
                    opacity=0.75,
                    error_y=dict(type='data', array=std_pos, visible=True, color='crimson')  # Corrigindo a cor aqui
))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Percentual de Operação Diária (POD) por mês - Serviço: LECD50 / 605", font=dict(color='black')),
    showlegend=False,
    yaxis_range=[0, 230],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valor médio e desvio-padrão mensais do POD',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        ),
        dict(
            x=0.445,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='antes do reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.745,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        ),
        dict(
            x=0.765,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='após o reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

# Adicionar linha horizontal para indicar o mínimo de 80%
fig.add_hline(y=80, annotation_text="min = 80%")

fig.show()

In [30]:
# Calculando a mediana, Q1 e Q3 para pod_pre_reprocessamento
median_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].median()
q1_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.25)
q3_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.75)

# Calculando a mediana, Q1 e Q3 para pod_pos_reprocessamento
median_pos = pod_pos_reprocessamento.groupby('mes_ano')['perc_km_planejada'].median()
q1_pos = pod_pos_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.25)
q3_pos = pod_pos_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.75)

In [31]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionando área sombreada para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=q1_pre.values, 
                         line=dict(width=0), fill=None, mode='lines', name='Q1 Pre'))
fig.add_trace(go.Scatter(x=median_pre.index, y=q3_pre.values, 
                         fill='tonexty', fillcolor='rgba(30,144,255,0.3)', mode='lines', name='Q3 Pre', 
                         line=dict(width=0))) 

# Adicionando a linha da mediana para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=median_pre.values,
                    mode='lines+markers',
                    name='Mediana Antes do Reprocessamento',
                    marker=dict(color='dodgerblue'),
                    opacity=0.75))

# Adicionando área sombreada para pod_pos_reprocessamento
fig.add_trace(go.Scatter(x=median_pos.index, y=q1_pos.values, 
                         line=dict(width=0), fill=None, mode='lines', name='Q1 Pos'))
fig.add_trace(go.Scatter(x=median_pos.index, y=q3_pos.values, 
                         fill='tonexty', fillcolor='rgba(220,20,60,0.3)', mode='lines', name='Q3 Pos', 
                         line=dict(width=0)))  # Aqui estamos tornando a linha Q3 invisível


# Adicionando a linha da mediana para pod_pos_reprocessamento
fig.add_trace(go.Scatter(x=median_pos.index, y=median_pos.values,
                    mode='lines+markers',
                    name='Mediana Após Reprocessamento',
                    marker=dict(color='crimson'),
                    opacity=0.75))


# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Percentual de Operação Diária (POD) por mês - Serviço: LECD50 / 605", font=dict(color='black')),
    showlegend=False,
    yaxis_range=[0, 150],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Mediana, 1º e 3º quartis mensais do POD',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        ),
        dict(
            x=0.40,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='antes do reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.70,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        ),
        dict(
            x=0.72,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='após o reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

# Adicionar linha horizontal para indicar o mínimo de 80%
fig.add_hline(y=80, annotation_text="min = 80%")

fig.show()

A explicação para a mediana = 0% em abril foi que o serviço LECD50 operou apenas em 12 dias do mês, enquanto em maio operou 21 dias.

### 4. Exportar planilha com dados

Exportar planilha resumo com valores a pagar antes e depois, assim como a km identificada na planilha de viagens completas.

In [32]:
km_mes_ano_soma = km_mes_ano[['distancia_total', 'distancia_total_reprocessada']].sum()
km_mes_ano_soma = km_mes_ano_soma.to_frame().T

km_mes_ano_soma = km_mes_ano_soma.rename(columns={
    'distancia_total': 'distancia_total_pre',
    'distancia_total_reprocessada': 'distancia_total_pos'
})
km_mes_ano_soma


tabela_subsidio_mes_soma = tabela_subsidio_mes[['viagens_pre', 'viagens_pos','valor_subsidio_pre', 'valor_subsidio_pos']].sum()
tabela_subsidio_mes_soma = tabela_subsidio_mes_soma.to_frame().T
tabela_subsidio_mes_soma

km_mes_ano_soma['key'] = 1
tabela_subsidio_mes_soma['key'] = 1

tabela_comparacao_exportar = km_mes_ano_soma.merge(tabela_subsidio_mes_soma, on='key', how='left').drop('key', axis=1)

tabela_comparacao_exportar
tabela_comparacao_exportar.to_excel("data/output/comparacao_pre_pos_reprocessamento_010.xlsx", index=False, engine='openpyxl')

OSError: Cannot save file into a non-existent directory: 'data\output'